In [0]:
!wget "https://www.machinehack.com/wp-content/uploads/2020/05/Financial_Risk_Participants_Data.zip"

--2020-05-16 06:22:47--  https://www.machinehack.com/wp-content/uploads/2020/05/Financial_Risk_Participants_Data.zip
Resolving www.machinehack.com (www.machinehack.com)... 107.180.71.20
Connecting to www.machinehack.com (www.machinehack.com)|107.180.71.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20635 (20K) [application/zip]
Saving to: ‘Financial_Risk_Participants_Data.zip’

Financial_Risk_Part 100%[===================>]  20.15K  --.-KB/s    in 0.1s    

2020-05-16 06:22:47 (142 KB/s) - ‘Financial_Risk_Participants_Data.zip’ saved [20635/20635]



In [0]:
!unzip Financial_Risk_Participants_Data.zip

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

In [0]:
train_df = pd.read_csv('Financial_Risk_Participants_Data/Train.csv')
train_df.head()

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results,IsUnderRisk
0,2,8.032,14,8,3,6,0,1
1,31,77.730,8,3,3,8,1,0
2,40,59.203,3,12,11,3,0,1
3,12,73.080,4,5,7,6,0,0
4,4,15.666,13,15,6,7,2,1


In [0]:
train_df.shape

(543, 8)

In [0]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543 entries, 0 to 542
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   City                  543 non-null    int64  
 1   Location_Score        543 non-null    float64
 2   Internal_Audit_Score  543 non-null    int64  
 3   External_Audit_Score  543 non-null    int64  
 4   Fin_Score             543 non-null    int64  
 5   Loss_score            543 non-null    int64  
 6   Past_Results          543 non-null    int64  
 7   IsUnderRisk           543 non-null    int64  
dtypes: float64(1), int64(7)
memory usage: 34.1 KB


In [0]:
train_df.describe()

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results,IsUnderRisk
count,543.000000,543.000000,543.000000,543.000000,543.000000,543.000000,543.000000,543.000000
mean,19.576427,32.259346,8.189687,7.327808,7.046041,5.530387,0.613260,0.626151
std,14.722687,24.887291,3.312022,3.450667,3.156978,1.839124,0.764509,0.484270
min,0.000000,5.185000,3.000000,3.000000,3.000000,3.000000,0.000000,0.000000
25%,6.000000,13.565000,6.000000,5.000000,5.000000,4.000000,0.000000,0.000000
50%,14.000000,19.803000,8.000000,7.000000,7.000000,6.000000,1.000000,1.000000
75%,37.000000,62.594000,11.000000,10.000000,8.000000,7.000000,1.000000,1.000000
max,44.000000,80.809000,15.000000,15.000000,15.000000,13.000000,10.000000,1.000000


In [0]:
x = train_df.drop(['IsUnderRisk'], axis=1)
x.head()

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results
0,2,8.032,14,8,3,6,0
1,31,77.730,8,3,3,8,1
2,40,59.203,3,12,11,3,0
3,12,73.080,4,5,7,6,0
4,4,15.666,13,15,6,7,2


In [0]:
y = train_df['IsUnderRisk']
y.head()

0    1
1    0
2    1
3    0
4    1
Name: IsUnderRisk, dtype: int64

In [0]:
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler().fit(x)  
transformer

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
x_norm = transformer.transform(x)

In [0]:
#x_norm, y
xgb = XGBClassifier(max_depth=6, n_estimators=145)
xgb.fit(x_norm, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=145, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
xgb.score(x_norm, y)

0.998158379373849

In [0]:
#predict_proba

In [0]:
test_df = pd.read_csv('Financial_Risk_Participants_Data/Test.csv')
test_df.head()

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results
0,41,18.272,13,12,9,7,0
1,17,64.799,6,10,7,4,1
2,31,68.890,3,3,7,8,0
3,3,16.492,15,10,7,4,1
4,9,17.178,7,3,8,3,1


In [0]:
x_pred = test_df.to_numpy()
x_pred_norm = transformer.transform(x_pred)

pred = xgb.predict_proba(x_pred_norm)

In [0]:
#pred[:,0]

In [0]:
submit = pd.DataFrame(columns=['0','1'])
submit['0'] = pred[:, 0]
submit['1'] = pred[:, 1]
submit.head()

,0,1
0,0.004533,0.995467
1,0.076691,0.923309
2,0.965599,0.034401
3,0.000769,0.999231
4,0.061220,0.938780


In [0]:
submit.to_excel('s5.xlsx', index=False)